In [80]:
include("../lib/Lib.jl")
include("../lib/DeepSpace.jl")

Main.DeepSpace

In [81]:
using DataFrames, DataFramesMeta
using Statistics
using ScikitLearn
using Main

@sk_import linear_model: LinearRegression
@sk_import linear_model: ElasticNet

PyObject <class 'sklearn.linear_model.coordinate_descent.ElasticNet'>

In [82]:
KEY = "2019chs_wk1"
YEAR = "2019"

"2019"

In [83]:
(headers, scores, teams) = Lib.read_stacked(KEY, YEAR)

(["adjustPoints", "autoPoints", "bay1", "bay2", "bay3", "bay4", "bay5", "bay6", "bay7", "bay8"  …  "techFoulCount", "teleopPoints", "topLeftRocketFar", "topLeftRocketNear", "topRightRocketFar", "topRightRocketNear", "totalPoints", "key", "level", "event"], 360×54 DataFrame. Omitted printing of 50 columns
│ Row │ adjustPoints │ autoPoints │ bay1          │ bay2          │
│     │ Int64⍰       │ Int64⍰     │ String⍰       │ String⍰       │
├─────┼──────────────┼────────────┼───────────────┼───────────────┤
│ 1   │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 2   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 3   │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 4   │ 0            │ 12         │ PanelAndCargo │ Panel         │
│ 5   │ 0            │ 15         │ Panel         │ PanelAndCargo │
│ 6   │ 0            │ 12         │ PanelAndCargo │ PanelAndCargo │
│ 7   │ 0            │ 15         │ PanelAndCargo │ PanelAndCargo │
│ 8   │ 0     

In [84]:
DeepSpace.calculated_columns!(scores)
scores[:ownPoints] = scores.totalPoints .- scores.foulPoints .- (3 .* scores.foulCount) .- (9 * scores.techFoulCount)

360-element Array{Int64,1}:
  62
  82
  72
  71
  66
  65
  52
  54
  62
  35
  59
 -25
  63
   ⋮
  20
  12
  70
  45
  59
  47
  57
  60
  74
  66
  51
  55

In [85]:
scores_train = @linq scores |> where(:level .== "qm")
teams_train = @linq teams |> where(:level .== "qm")

#scores_train = scores
#teams_train = teams

scores_test = @linq scores |> where(:level .!= "qm")
teams_test = @linq teams |> where(:level .!= "qm")

(ß, teamnums, teamindex) = Lib.opr_ß(scores_train, teams_train ; shortevent = true)

([0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 1.0 0.0 … 0.0 0.0], Union{Missing, String}["frc1080", "frc1086", "frc1123", "frc116", "frc122", "frc1262", "frc1413", "frc1418", "frc1522", "frc1598"  …  "frc6504", "frc6543", "frc6584", "frc6802", "frc6863", "frc6882", "frc7330", "frc7429", "frc888", "frc977"], Dict("frc888"=>74,"frc1086"=>2,"frc2421"=>24,"frc3359"=>34,"frc3941"=>41,"frc5546"=>51,"frc4242"=>44,"frc1418"=>8,"frc3136"=>30,"frc6802"=>69…))

In [86]:
model = LinearRegression(fit_intercept=false)
components = DataFrame([teamnums], [:team])

for col in Lib.numeric_col_names(scores_train)
    y = convert.(Float64, coalesce(scores_train[col], -1.0))
    modelfit = ScikitLearn.fit!(model, ß, y)
    components[col] = modelfit[:coef_]
end
components

,team,adjustPoints,autoPoints,cargoPoints,completeRocketRankingPoint,completedRocketFar,completedRocketNear,foulCount,foulPoints,habClimbPoints,habDockingRankingPoint,hatchPanelPoints,rp,sandStormBonusPoints,techFoulCount,teleopPoints,totalPoints,rocketLowPoints,rocketMidPoints,rocketTopPoints,rocketPoints,ownPoints
,String⍰,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,frc1080,0.0,3.02345,4.04895,-0.014973,-0.0159382,0.0,-0.156126,-0.369823,3.35403,0.0469256,0.663347,0.199775,3.02345,-0.0150174,8.06633,10.72,1.06021,0.745208,0.590239,2.39566,11.6933
2,frc1086,0.0,3.42255,4.54003,-0.00639716,0.00695878,0.0,-0.0874686,0.172497,3.32256,-0.0209212,0.711476,-0.0585473,3.42255,-0.0219442,8.57406,12.1691,-0.327525,-0.449393,-0.229599,-1.00652,12.4565
3,frc1123,0.0,3.32659,1.24485,0.00599277,1.38778e-17,0.0,0.501704,0.111629,2.06334,0.0638652,-0.0481814,0.0748255,3.32659,-0.0591353,3.26001,6.69822,0.816934,-0.0230095,-0.041122,0.752803,5.6137
4,frc116,0.0,3.22058,6.32379,-0.0149377,-2.08167e-17,0.0,0.380765,-0.957658,4.5475,0.258188,0.358487,0.604582,3.22058,0.0782482,11.2298,13.4927,0.934338,-0.526824,0.0548587,0.462373,12.6038
5,frc122,0.0,3.99446,5.02112,-0.0197229,-0.0140204,0.0,-0.0322353,0.13493,4.00184,0.0275384,1.99484,0.293968,3.99446,0.0427331,11.0178,15.1472,0.34146,-0.135938,-0.161916,0.0436063,14.7244
6,frc1262,0.0,3.36678,8.67643,0.0894812,0.0850349,0.0,0.259097,0.0798699,5.01688,0.108632,6.14141,1.06552,3.36678,-0.0358919,19.8347,23.2814,5.67182,3.46605,2.63103,11.7689,22.7472
7,frc1413,0.0,3.09664,6.29119,-0.000688515,-0.00246549,0.0,-0.202185,1.81764,5.05944,0.175867,1.77831,0.778047,3.09664,-0.0262021,13.1289,18.0432,1.51251,0.977214,-0.0293405,2.46038,17.068
8,frc1418,0.0,2.64771,6.22739,0.0902238,6.93889e-18,0.0,0.323051,-0.0856498,4.42805,0.187909,9.00321,0.885723,2.64771,0.0514966,19.6586,22.2207,5.31529,4.70538,5.28673,15.3074,20.8737
9,frc1522,0.0,1.73207,3.01897,0.00622271,0.00321153,0.0,0.0347033,1.90515,-0.0535299,-0.13141,0.980086,0.206705,1.73207,-0.0282698,3.94552,7.58274,-0.105731,-0.301557,-0.134337,-0.541625,5.82791


In [89]:
colname = [:ownPoints, :rocketTopPoints, :rocketMidPoints, :rocketLowPoints, :cargoPoints, :team]
sort(components, colname, rev=true)[colname]

,ownPoints,rocketTopPoints,rocketMidPoints,rocketLowPoints,cargoPoints,team
,Float64,Float64,Float64,Float64,Float64,String⍰
1,36.3876,-0.290303,-0.670676,6.22786,11.867,frc346
2,27.0536,0.0908654,0.907993,0.569468,11.5936,frc614
3,25.4732,-0.254496,-0.797741,-0.28441,8.59081,frc612
4,22.7472,2.63103,3.46605,5.67182,8.67643,frc1262
5,22.605,0.127404,0.0349564,0.662693,9.77812,frc2998
6,22.0823,-0.567204,0.488676,5.0565,8.79354,frc2534
7,21.0733,0.150021,1.36221,4.40351,5.59598,frc619
8,20.8737,5.28673,4.70538,5.31529,6.22739,frc1418
9,20.516,0.0664766,0.862118,2.16545,9.02606,frc6802


In [88]:
using CSV
CSV.write("../output/$(KEY)_components.csv", components)

"../output/2019chs_wk1_components.csv"